<a href="https://colab.research.google.com/github/https-deeplearning-ai/tensorflow-2-public/blob/adding_C2/C2/W1/ungraded_labs/C2_W1_Lab_1_Linear_Regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

##### Copyright 2018 The TensorFlow Authors.

In [2]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Running TFLite Models

## Setup

In [3]:
try:
    %tensorflow_version 2.x
except:
    pass

In [4]:
import pathlib
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf

print('\u2022 Using TensorFlow Version:', tf.__version__)

• Using TensorFlow Version: 2.16.1


## Create a Basic Model of the Form y = mx + c

In [5]:
# Create a simple Keras model.
x = np.array([-1, 0, 1, 2, 3, 4])
y = np.array([-3, -1, 1, 3, 5, 7])

model = tf.keras.models.Sequential([
        tf.keras.layers.Dense(units=1, input_shape=[1])
])

model.compile(optimizer='sgd',
              loss='mean_squared_error')

model.fit(x, y, epochs=200)

Epoch 1/200


c:\Python312\Lib\site-packages\keras\src\layers\core\dense.py:88: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 477ms/step - loss: 53.5095
Epoch 2/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step - loss: 42.5060
Epoch 3/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 164ms/step - loss: 33.8406
Epoch 4/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step - loss: 27.0149
Epoch 5/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step - loss: 21.6367
Epoch 6/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step - loss: 17.3976
Epoch 7/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 163ms/step - loss: 14.0547
Epoch 8/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step - loss: 11.4172
Epoch 9/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 89ms/step - loss: 9.3347
Epoch 10/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 157ms/step - loss: 7.6891
Epoch 11/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step - loss: 6.3873
Epoch 12/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step - loss: 5.3562
Epoch 13/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step - loss: 4.5381
Epoch 14/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step - loss: 3.8879
Epoch 15/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - loss: 3.3698
Epoch 16/200
1/1 ━━

## Generate a SavedModel

In [8]:
!pip show wrapt

Name: wrapt
Version: 1.14.1
Summary: Module for decorators, wrappers and monkey patching.
Home-page: https://github.com/GrahamDumpleton/wrapt
Author: Graham Dumpleton
Author-email: Graham.Dumpleton@gmail.com
License: BSD
Location: C:\Python312\Lib\site-packages
Requires: 
Required-by: tensorflow-datasets, tensorflow-intel


In [7]:
export_dir = 'saved_model/1'
tf.saved_model.save(model, export_dir)

TypeError: this __dict__ descriptor does not support '_DictWrapper' objects

## Convert the SavedModel to TFLite

In [ ]:
# Convert the model.
converter = tf.lite.TFLiteConverter.from_saved_model(export_dir)
tflite_model = converter.convert()

OSError: SavedModel file does not exist at: savedmodel.h5\{saved_model.pbtxt|saved_model.pb}

In [ ]:
tflite_model_file = pathlib.Path('model.tflite')
tflite_model_file.write_bytes(tflite_model)

## Initialize the TFLite Interpreter To Try It Out

In [ ]:
# Load TFLite model and allocate tensors.
interpreter = tf.lite.Interpreter(model_content=tflite_model)
interpreter.allocate_tensors()

# Get input and output tensors.
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

In [ ]:
# Test the TensorFlow Lite model on random input data.
input_shape = input_details[0]['shape']
inputs, outputs = [], []
for _ in range(100):
    input_data = np.array(np.random.random_sample(input_shape), dtype=np.float32)
    interpreter.set_tensor(input_details[0]['index'], input_data)
    
    interpreter.invoke()
    tflite_results = interpreter.get_tensor(output_details[0]['index'])
    
    # Test the TensorFlow model on random input data.
    tf_results = model(tf.constant(input_data))
    output_data = np.array(tf_results)
    
    inputs.append(input_data[0][0])
    outputs.append(output_data[0][0])

## Visualize the Model

In [ ]:
%matplotlib inline

plt.plot(inputs, outputs, 'r')
plt.show()

## Download the TFLite Model File

If you are running this notebook in a Colab, you can run the cell below to download the tflite model to your local disk. 

**Note**: If the file does not download when you run the cell, try running the cell a second time.

In [ ]:
try:
    from google.colab import files
    files.download(tflite_model_file)
except:
    pass